In [2]:
import pyspark
from delta import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
spark = (SparkSession.builder.appName("SparkSample").getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/12 01:50:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/12 01:50:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/12 01:50:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Read in a single file

In [3]:
df = spark.read.text("textfile.txt")

create a temporary view in spark

In [14]:
df.createOrReplaceTempView("vw_temp_view")

In [18]:
spark.sql("select * from vw_temp_view").show(2)

+--------------------+
|               value|
+--------------------+
|Disclaimer: While...|
|                    |
+--------------------+
only showing top 2 rows



create a view using SQL that will persist

In [21]:
spark.sql("""create table tbl_view as select * from vw_temp_view""")

25/02/12 01:59:42 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
25/02/12 01:59:42 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/02/12 01:59:42 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
25/02/12 01:59:42 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/02/12 01:59:42 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/02/12 01:59:42 WARN HiveMetaStore: Location: file:/user/hive/warehouse/tbl_view specified for non-external table:tbl_view
chgrp: changing ownership of 'file:///user/hive/warehouse/tbl_view': chown: changing group of '/user/hive/warehouse/tbl_view': Operation not permitted


DataFrame[]

In [22]:
spark.sql("""create table vw_view as select * from tbl_view""")

25/02/12 02:00:13 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
25/02/12 02:00:13 WARN HiveMetaStore: Location: file:/user/hive/warehouse/vw_view specified for non-external table:vw_view
chgrp: changing ownership of 'file:///user/hive/warehouse/vw_view': chown: changing group of '/user/hive/warehouse/vw_view': Operation not permitted


DataFrame[]

CTE (Common Table Expression) is just a fancy query

In [24]:
spark.sql("""with cte_one as (
    select * from vw_view
    )
    select * from cte_one""").show(2)

+--------------------+
|               value|
+--------------------+
|Disclaimer: While...|
|                    |
+--------------------+
only showing top 2 rows



In [11]:
spark.sql("DESCRIBE FORMATTED tbl_view").show(truncate=False)

+----------------------------+----------------------------------------------------------+-------+
|col_name                    |data_type                                                 |comment|
+----------------------------+----------------------------------------------------------+-------+
|value                       |string                                                    |NULL   |
|                            |                                                          |       |
|# Detailed Table Information|                                                          |       |
|Catalog                     |spark_catalog                                             |       |
|Database                    |default                                                   |       |
|Table                       |tbl_view                                                  |       |
|Owner                       |sparkuser                                                 |       |
|Created Time       